<a href="https://colab.research.google.com/github/lucastrschneider/MAP3121/blob/main/ep1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício Programa 1 - MAP3121

Jean Carlos Mello Xavier Faria - 11259628

Lucas Tonini Rosenberg Schneider - 11260850

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Método QR

In [8]:
class QRMethod:
  def __init__(self, a_alfa, a_beta, spectral=True, print_steps=False, print_matrices=False):
    self.n = a_alfa.shape[0]
    self.spectral = spectral
    self.k = 0
    self.print_steps = print_steps
    self.print_matrices = print_matrices

    assert a_alfa.shape[0] == self.n
    assert a_beta.shape[0] == self.n-1

    self.last_alfa = np.ndarray(self.n, dtype=float)
    self.last_beta = np.ndarray(self.n-1, dtype=float)
    self.actual_alfa = np.ndarray(self.n, dtype=float)
    self.actual_beta = np.ndarray(self.n-1, dtype=float)
    self.auto_vec = np.identity(self.n, dtype=float)

    np.copyto(self.actual_alfa, a_alfa)
    np.copyto(self.actual_beta, a_beta)

  def _wilkinson_coeficient(self):
    pass

  def _givens_coeficients(self, k):
    alfa = self.actual_alfa[k]
    beta = self.actual_beta[k]

    # Método normal
    div = np.sqrt(alfa**2 + beta**2)
    ck = alfa / div
    sk = -beta / div

    # Método mais estável
    # if abs(alfa) > abs(beta):
    #   tau = -beta/alfa
    #   ck = 1 / np.sqrt(1 + tau ** 2)
    #   sk = ck * tau
    # else:
    #   tau = -alfa/beta
    #   sk = 1 / np.sqrt(1 + tau ** 2)
    #   ck = sk * tau
    
    if self.print_steps:
      print(f'{(ck, sk)}')
    return (ck, sk)

  def _get_R(self, gama):
    R = np.zeros((self.n, self.n))
    for i in range(self.n-1):
      R[i,i] = self.actual_alfa[i]
      R[i,i+1] = gama[i]

    R[self.n-1, self.n-1] = self.actual_alfa[self.n-1]

    return R

  def get_A(self):
    A = np.zeros((self.n, self.n))
    for i in range(self.n-1):
      A[i,i] = self.actual_alfa[i]
      A[i,i+1] = self.actual_beta[i]
      A[i+1,i] = self.actual_beta[i]
    A[self.n-1, self.n-1] = self.actual_alfa[self.n-1]

    return A

  def get_Qi(self, i, ci, si):
    Qi = np.identity(self.n)
    Qi[i,i] = ci
    Qi[i+1,i+1] = ci
    Qi[i,i+1] = -si
    Qi[i+1,i] = si

    return Qi

  def get_Q(self, ck, sk):
    Q = np.identity(self.n)
    for i in range(self.n-1):
      Q = np.matmul(Q, self.get_Qi(i, ck[i], sk[i]).T)

    return Q

  def _iterate_once(self, epsilon):

    if self.print_matrices:
      Ai = self.get_A()

    # Euristica de Wilkinson (coeficiente uk)
    if self.spectral and self.k > 0:
      uk = self._wilkinson_coeficient()
    else:
      uk = 0
    

    np.copyto(self.last_alfa, self.actual_alfa)
    np.copyto(self.last_beta, self.actual_beta)
    gama = self.actual_beta.copy()

    self.actual_alfa -= uk

    # Coeficientes de Givens (ck e sk)
    ck = np.ndarray(self.n-1, dtype=float)
    sk = np.ndarray(self.n-1, dtype=float)
    
    if self.print_steps:
      print(f'\n\nIteração {self.k+1}:')
      print(f'\nDeslocamento espectral: uk = {uk}')
      print(f'\nCoeficientes de Givens:')
      

    # Alfa e beta da matriz R(k)
    for i in range(self.n-1):
      # Calculo dos coefientes da iteração atual
      ck[i], sk[i] = self._givens_coeficients(i)

      alfa_i = self.actual_alfa[i] * ck[i] - self.actual_beta[i] * sk[i]
      gama_i = gama[i] * ck[i] - self.actual_alfa[i+1] * sk[i]
      alfa_j = gama[i] * sk[i] + self.actual_alfa[i+1] * ck[i]

      self.actual_alfa[i] = alfa_i
      gama[i] = gama_i
      self.actual_alfa[i+1] = alfa_j

      if i < self.n-2:
        gama[i+1] = gama[i+1] * ck[i]

    if self.print_steps:
      print(f'\nMatriz R')
      print(f'Deslocamento espectral: uk = {uk}')
      print(f'Coeficientes de Givens:')

    if self.print_matrices:
      Ri = self._get_R(gama)

    # Alfa e beta da matriz A(k+1)
    for i in range(self.n-1):
      self.actual_alfa[i] = self.actual_alfa[i] * ck[i] - gama[i] * sk[i]
      self.actual_beta[i] = -self.actual_alfa[i+1] * sk[i]
      self.actual_alfa[i+1] = self.actual_alfa[i+1] * ck[i]

      print(self.actual_alfa[i], self.actual_beta[i])
      print(self.actual_alfa[i+1])

    # Correção por deslocamente espectral
    self.actual_alfa += uk

    if self.print_matrices:
      Ai_plus_1 = self.get_A()
      Qi = self.get_Q(ck, sk)
    
    # Atualiza autovetores

    self.k += 1

    if self.print_matrices:
      print(Ai)
      print(Ri)
      print(Qi)
      print(Ai_plus_1)
    # return (Ai, Ri, Qi, Ai_plus_1)

  def iterate(self, epsilon):
    pass


In [9]:
alfa = 4*np.ones(3)
beta = 3*np.ones(2)

method = QRMethod(alfa, beta, False, True, True)

In [10]:
method._iterate_once(0)


Iteration 1:
Deslocamento espectral: uk = 0
Coeficientes de Givens:
(0.8, -0.6)
(0.422885465331124, -0.9061831399952655)
6.880000000000001 1.9863534428696221
2.648471257159496
5.324379562043797 -0.437956204379562
-0.20437956204379565
[[4. 3. 0.]
 [3. 4. 3.]
 [0. 3. 4.]]
[[ 5.          4.8         0.        ]
 [ 0.          3.31058907  4.63965768]
 [ 0.          0.         -0.48329767]]
[[ 0.8        -0.25373128  0.54370988]
 [ 0.6         0.33830837 -0.72494651]
 [ 0.          0.90618314  0.42288547]]
[[ 6.88        1.98635344  0.        ]
 [ 1.98635344  5.32437956 -0.4379562 ]
 [ 0.         -0.4379562  -0.20437956]]
